In [ ]:
pip install pandera

In [ ]:
pip install gcsfs

In [ ]:
import os
import pandas as pd
from google.cloud import storage

In [ ]:
serviceAccount = '/content/Chave Ingestao Apache.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = serviceAccount

In [ ]:
#leitura do arquivo em JSON
df = pd.read_json(r'gs://projeto-final-grupo09/entrada_dados/Projeto Final', lines = True)
df.head(1)

In [ ]:
#Renomeando colunas
df.rename(columns={'id':'identificacao','created_on':'criado_em','operation':'operacao','property_type':'tipo_propriedade',
                   'place_name':'nome_do_local','place_with_parent_names':'pais_local','country_name':'pais','state_name':'estado',
                   'geonames_id':'g_nomes','lat_lon':'latitude_longitude','lat':'latitude','lon':'longitude','price':
                   'preco_cheio','currency':'moeda','price_aprox_local_currency':'preco',
                   'price_aprox_usd':'preco_aproximado_dolar','surface_total_in_m2':'area_total_por_m2',
                   'surface_covered_in_m2':'area_construcao_em_m2','price_usd_per_m2':'preco_dolar_por_m2',
                   'price_per_m2':'preco_por_m2','floor':'andar','rooms':'quartos','expenses':'despesas',
                   'properati_url':'url_da_propriedade','description':'descricao', 'title':'titulo',
                   'image_thumbnail':'miniatura_imagem'}, inplace = True)
df.head(2)


In [ ]:
#chamar a coluna de operacao para ver se tem algo além de venda (no caso a coluna só tem sell - venda - entao irei dropar posteriormente)
sorted(pd.unique(df['operacao']))

['sell']

In [ ]:
#chamar a coluna de país para ver se tem algo além de Brasil (no caso só tem Brasil - então iremos dropar posteriormente)
sorted(pd.unique(df['pais']))

['Brasil']

In [ ]:
#chamar a coluna de moeda para ver se tem algo além de BRL (no caso só tem BRL - então iremos dropar posteriormente)
sorted(pd.unique(df['moeda']))

['', 'BRL', 'COP']

In [ ]:
#criacao de variavel - colunas - para posterior drop
colunas = ['operacao', 'pais', 'moeda', 'latitude_longitude', 'latitude', 'longitude', 'preco_aproximado_dolar', 'pais_local', 
'preco_dolar_por_m2', 'andar', 'despesas', 'descricao', 'titulo', 'miniatura_imagem', 'url_da_propriedade', 'preco_cheio']
df.drop(colunas, axis=1, inplace=True)

In [ ]:
#verificando se há (e quantos são) os valores na coluna nome_do_local
df['nome_do_local'].value_counts()

Porto Alegre       14823
Praia Grande       14425
Centro             13970
Santo André        12884
Copacabana          5534
                   ...  
Jardim Cerrado         1
Vila Josefina          1
Caju                   1
Jardim Mosteiro        1
Jardim Monjolo         1
Name: nome_do_local, Length: 4400, dtype: int64

In [ ]:
#verificar se há apenas um valor na coluna (no caso a coluna propriedade_tipo tem 3 informações significativas e uma (PH) que será dropada)
sorted(pd.unique(df['tipo_propriedade']))

['PH', 'apartment', 'house', 'store']

In [ ]:
#contando quantos valores tem cada um dos itens em tipo_propriedade - casa, apartamento e lojas
df['tipo_propriedade'].value_counts()

apartment    273577
house        135213
store          8894
PH              429
Name: tipo_propriedade, dtype: int64

In [ ]:
#traduzindo as informações contidas na coluna de tipo_propriedade
df['tipo_propriedade'].replace(['house', 'apartment', 'store'],['casa','apartamento','loja'], inplace = True)

In [ ]:
#quantidade de quartos e salas (no caso das lojas)
df['quartos'].value_counts()

In [ ]:
#chamar a coluna de quartos para descobrir quais os valores contidos
sorted(pd.unique(df['quartos']))

In [ ]:
#devido a coluna quartos ser um float, forçamos ele a se tornar um numero inteiro e o NaN se tornar 0
df['quartos'] = df['quartos'].fillna(0.0).astype(int)

In [ ]:
df.head(10)

In [ ]:
df.to_csv("gs://lucao-buck", sep=",", index=False)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')